#### Pyspark Handling Missing Values
- Dropping Columns
- Dropping Rows
- Various Parameter in Dropping functionalities
- Handling Missing values by Mean, Median, Mode

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('HMValues').getOrCreate()

In [6]:
df = spark.read.csv('people.csv', header=True, inferSchema=True)

In [7]:
df.show()

+---------+---+----------+------+------+
|     Name|Age|Experience|Gender|Salary|
+---------+---+----------+------+------+
|    Peter| 45|        15|     M|  5000|
|Clevelend| 42|         3|  NULL|  1000|
| Quagmire| 61|        40|     M|  3500|
|      Joe| 40|         5|  NULL|  NULL|
|     Lois| 40|         4|     F|  2500|
|      Meg| 17|         1|     F|  NULL|
+---------+---+----------+------+------+



In [8]:
## dropping the columns
df.drop('Age').show() ## or ['Age', 'Name'] for multiple columns

+---------+----------+------+------+
|     Name|Experience|Gender|Salary|
+---------+----------+------+------+
|    Peter|        15|     M|  5000|
|Clevelend|         3|  NULL|  1000|
| Quagmire|        40|     M|  3500|
|      Joe|         5|  NULL|  NULL|
|     Lois|         4|     F|  2500|
|      Meg|         1|     F|  NULL|
+---------+----------+------+------+



In [9]:
df.show()

+---------+---+----------+------+------+
|     Name|Age|Experience|Gender|Salary|
+---------+---+----------+------+------+
|    Peter| 45|        15|     M|  5000|
|Clevelend| 42|         3|  NULL|  1000|
| Quagmire| 61|        40|     M|  3500|
|      Joe| 40|         5|  NULL|  NULL|
|     Lois| 40|         4|     F|  2500|
|      Meg| 17|         1|     F|  NULL|
+---------+---+----------+------+------+



In [10]:
df.na.drop().show() ## drops every null value

+--------+---+----------+------+------+
|    Name|Age|Experience|Gender|Salary|
+--------+---+----------+------+------+
|   Peter| 45|        15|     M|  5000|
|Quagmire| 61|        40|     M|  3500|
|    Lois| 40|         4|     F|  2500|
+--------+---+----------+------+------+



In [12]:
df.na.drop(how="any").show() ## any used to drop records that contain at least one null

+--------+---+----------+------+------+
|    Name|Age|Experience|Gender|Salary|
+--------+---+----------+------+------+
|   Peter| 45|        15|     M|  5000|
|Quagmire| 61|        40|     M|  3500|
|    Lois| 40|         4|     F|  2500|
+--------+---+----------+------+------+



In [13]:
df.na.drop(how="all").show() ## all used to drop records where every columns are null

+---------+---+----------+------+------+
|     Name|Age|Experience|Gender|Salary|
+---------+---+----------+------+------+
|    Peter| 45|        15|     M|  5000|
|Clevelend| 42|         3|  NULL|  1000|
| Quagmire| 61|        40|     M|  3500|
|      Joe| 40|         5|  NULL|  NULL|
|     Lois| 40|         4|     F|  2500|
|      Meg| 17|         1|     F|  NULL|
+---------+---+----------+------+------+



In [16]:
## threshhold
df.na.drop(how="any", thresh=4).show() ## thresh will keep records that have at least n not-null values

+---------+---+----------+------+------+
|     Name|Age|Experience|Gender|Salary|
+---------+---+----------+------+------+
|    Peter| 45|        15|     M|  5000|
|Clevelend| 42|         3|  NULL|  1000|
| Quagmire| 61|        40|     M|  3500|
|     Lois| 40|         4|     F|  2500|
|      Meg| 17|         1|     F|  NULL|
+---------+---+----------+------+------+



In [17]:
## subset
df.na.drop(how="any", subset=['Gender']).show() ## subset is used to drop null values in specific column

+--------+---+----------+------+------+
|    Name|Age|Experience|Gender|Salary|
+--------+---+----------+------+------+
|   Peter| 45|        15|     M|  5000|
|Quagmire| 61|        40|     M|  3500|
|    Lois| 40|         4|     F|  2500|
|     Meg| 17|         1|     F|  NULL|
+--------+---+----------+------+------+



In [20]:
### filling the missing value
df.na.fill(0, 'Salary').show()

+---------+---+----------+------+------+
|     Name|Age|Experience|Gender|Salary|
+---------+---+----------+------+------+
|    Peter| 45|        15|     M|  5000|
|Clevelend| 42|         3|  NULL|  1000|
| Quagmire| 61|        40|     M|  3500|
|      Joe| 40|         5|  NULL|     0|
|     Lois| 40|         4|     F|  2500|
|      Meg| 17|         1|     F|     0|
+---------+---+----------+------+------+



In [29]:
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols = ['Salary'],
    outputCols = ["{}_imputed".format(c) for c in ['Salary']]
).setStrategy("mean") ## use median, mode

In [30]:
imputer.fit(df).transform(df).show()

+---------+---+----------+------+------+--------------+
|     Name|Age|Experience|Gender|Salary|Salary_imputed|
+---------+---+----------+------+------+--------------+
|    Peter| 45|        15|     M|  5000|          5000|
|Clevelend| 42|         3|  NULL|  1000|          1000|
| Quagmire| 61|        40|     M|  3500|          3500|
|      Joe| 40|         5|  NULL|  NULL|          3000|
|     Lois| 40|         4|     F|  2500|          2500|
|      Meg| 17|         1|     F|  NULL|          3000|
+---------+---+----------+------+------+--------------+

